In [1]:
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use("grayscale")

import torch.nn as nn

In [2]:
DATA_DIR = "/Users/tianyudu/Documents/UToronto/Course/ECO499/ugthesis/data/ready_to_use/"

In [3]:
def load_data(
    path: str,
    start = datetime(2000, 1, 1),
    end = datetime(2019, 9, 30)
) -> pd.DataFrame:
    def core(name: str):
        df = pd.read_csv(
            path + name,
            index_col="DATE",
            date_parser=lambda x: datetime.strptime(x, "%Y-%m-%d")
        ).asfreq("B")
        df.fillna(pd.NA)
        df = df[
            (start <= df.index) & (df.index <= end)
        ]
        return df
    df_returns = core("returns_filled.csv")
    df_rpna = core("rpna.csv")
    return df_returns, df_rpna

In [7]:
df_returns, df_rpna = load_data(DATA_DIR)
df_rpna.fillna(0, inplace=True)
df = pd.concat([df_returns, df_rpna], axis=1)
df.head()

,RETURN,ESS_MEAN,ESS_TOTAL,WESS_MEAN,WESS_TOTAL,NUM_EVENTS,NUM_POSITIVE_ESS,NUM_NEGATIVE_ESS,NUM_NEUTRAL_ESS,NUM_POSITIVE_WESS,NUM_NEGATIVE_WESS,NUM_NEUTRAL_WESS
DATE,,,,,,,,,,,,
2000-01-03,NaN,18.0,36.0,15.7500,31.50,2.0,2.0,0.0,0.0,2.0,0.0,0.0
2000-01-04,-0.036252,5.0,30.0,3.5550,21.33,6.0,3.0,2.0,1.0,3.0,2.0,1.0
2000-01-05,-0.036252,-6.0,-12.0,-6.0000,-12.00,2.0,0.0,1.0,1.0,0.0,1.0,1.0
2000-01-06,0.005663,-26.5,-53.0,-26.5000,-53.00,2.0,0.0,2.0,0.0,0.0,2.0,0.0
2000-01-07,0.000000,-15.0,-60.0,-10.7625,-43.05,4.0,0.0,4.0,0.0,0.0,4.0,0.0


# Wavenet

In [8]:
import torch
import math

In [9]:
class Conv(torch.nn.Module):
    """
    A convolution with the option to be causal and use xavier initialization
    """
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int=1,
        stride: int=1,
        dilation: int=1,
        bias: bool=True,
        w_init_gain: str="linear",
        is_causal: bool=True
    ) -> None:
        super(Conv, self).__init__()
        self.is_causal = is_causal
        self.kernel_size = kernel_size
        self.dilation = dilation

        self.conv = torch.nn.Conv1d(
            in_channels, out_channels,
            kernel_size=kernel_size,
            stride=stride,
            dilation=dilation,
            bias=bias
        )

        torch.nn.init.xavier_uniform(
            self.conv.weight, 
            gain=torch.nn.init.calculate_gain(w_init_gain)
        )

    def forward(self, signal):
        if self.is_causal:
            padding = (int((self.kernel_size - 1) * (self.dilation)), 0)
                signal = torch.nn.functional.pad(signal, padding) 
        return self.conv(signal)